In [21]:
import torch
import torch.nn as nn
import torch.functional as F
import pandas as pd
from tqdm import tqdm
import numpy as np
import cv2 as cv

class SteeringNet(nn.Module):
    def __init__(self):
        super(SteeringNet, self).__init__()
        
        self.fc0 = nn.Linear(in_features=1, out_features=64)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=64*25*25, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=3)
        self.relu = nn.ReLU()
        
    def forward(self, x, param):
        param = self.fc0(param)
        #x = torch.cat((x, param.view(param.size(0), 1, 1, 1).repeat(1, 1, x.size(2), x.size(3))), dim=1)
        x = torch.cat((x, param.view(param.size(0), -1, 1, 1).repeat(1, 1, x.size(2), x.size(3))), dim=1)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.pool3(x)
        x = x.view(-1, 64*25*25)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

base_path = '../dane/img_zaznaczone/'
IMAGE_HEIGHT = 100
IMAGE_WIDTH = 100

In [138]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 6 * 6, 512)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 1)
        self.bn = nn.BatchNorm1d(512)
    
    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.pool1(x)
        x = self.conv3(x)
        x = nn.ReLU()(x)
        x = self.conv4(x)
        x = nn.ReLU()(x)
        x = self.pool2(x)
        x = self.conv5(x)
        x = nn.ReLU()(x)
        x = self.pool3(x)
        x = self.conv6(x)
        x = nn.ReLU()(x)
        x = self.pool4(x)
        x = x.view(-1, 128 * 6 * 6)
        x = self.bn(self.fc1(x))
        x = nn.ReLU()(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = nn.ReLU()(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = nn.ReLU()(x)
        x = self.fc4(x)
        return x
    
model = Net().cuda()

In [71]:
log = pd.read_csv(base_path + 'img/driving_log.csv')
log.head(1)

,center,left,right,steering,throttle,brake,speed
0,IMG/center_2016_12_01_13_30_48_287.jpg,IMG/left_2016_12_01_13_30_48_287.jpg,IMG/right_2016_12_01_13_30_48_287.jpg,0.0,0.0,0.0,22.14829


In [59]:
import matplotlib.pyplot as plt
import skimage.io as io
from skimage.color import rgb2gray
img = cv.imread(base_path + 'IMG/center_2016_12_01_13_30_48_287.jpg')
#io.imshow(img[:, :, 1] > 20)

In [60]:
def random_flip(image, steering_angle):
    
    image = cv.flip(image, 1)
    steering_angle = -steering_angle
    
    return image, steering_angle

In [61]:
from sklearn.model_selection import train_test_split

In [198]:
train_x = []
train_y = []
test_x = []
test_y = []

n = log.shape[0]
for i in tqdm(range(n)):
    img = cv.imread(base_path + log['center'][i])
    
    img = cv.cvtColor(img, cv.COLOR_RGB2HSV)[:,:,1]
    img = cv.resize(img,(IMAGE_HEIGHT,IMAGE_WIDTH))
    img = img / 255.0
        
    angle = log['steering'][i]
    speed = log['speed'][i]
    train_x.append(img)
    train_y.append(angle)
    img, angle = random_flip(img, angle)
    train_x.append(img)
    train_y.append(angle)

100%|█████████████████████████████████████████████████████████████████████████████| 8036/8036 [00:28<00:00, 277.30it/s]


In [199]:
train_x = np.array(train_x)
train_x = np.reshape(train_x, [train_x.shape[0], train_x.shape[1], train_x.shape[2], 1])
train_y = np.array(train_y)
train_y = np.reshape(train_y, [train_y.shape[0], 1])

In [200]:
x_train,x_test,y_train,y_test = train_test_split(train_x,train_y,random_state=42,test_size=.20)
print(type(x_test), x_test.shape)
print(type(y_test), y_test.shape)

<class 'numpy.ndarray'> (3215, 100, 100, 1)
<class 'numpy.ndarray'> (3215, 1)


In [201]:
model = model.train()
model = model.cuda()
x_train_tensor = torch.from_numpy(x_train).cuda()
y_train_tensor = torch.from_numpy(y_train).cuda()
x_test_tensor = torch.from_numpy(x_test).cuda()
y_test_tensor = torch.from_numpy(y_test).cuda()

In [204]:
x = torch.randn(4, 1, 100, 100).cuda()
output = model(x)
output.shape
output

tensor([[0.0063],
        [0.0086],
        [0.0239],
        [0.0105]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [209]:
BATCH_SIZE = 100
EPOCHS = 8

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_x), BATCH_SIZE)):
        batch_X = x_train_tensor[i:i+BATCH_SIZE].view(-1, 1, 100, 100).to(torch.float32)
        batch_y = y_train_tensor[i:i+BATCH_SIZE].to(torch.float32)

        opt.zero_grad()
        outputs = model(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        opt.step()
    print(loss)
    
print(batch_X.shape)
print(batch_y.shape)
#print(io.imshow(np.array(batch_X[31].squeeze().cpu())))
#print(batch_X[31][0][50][50])

100%|████████████████████████████████████████████████████████████████████████████████| 161/161 [00:03<00:00, 42.23it/s]


tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)


100%|████████████████████████████████████████████████████████████████████████████████| 161/161 [00:03<00:00, 44.93it/s]


tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)


100%|████████████████████████████████████████████████████████████████████████████████| 161/161 [00:03<00:00, 44.76it/s]


...

tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)


100%|████████████████████████████████████████████████████████████████████████████████| 161/161 [00:03<00:00, 45.11it/s]


tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)


100%|████████████████████████████████████████████████████████████████████████████████| 161/161 [00:03<00:00, 44.11it/s]

tensor(nan, device='cuda:0', grad_fn=<MseLossBackward0>)
torch.Size([0, 1, 100, 100])
torch.Size([0, 1])


In [190]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 12 * 12, 256)
        self.fc2 = nn.Linear(256, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)), (2, 2))
        x = self.pool(F.relu(self.conv2(x)), (2, 2))
        x = self.pool(F.relu(self.conv3(x)), (2, 2))
        x = x.view(-1, 128*12*12)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net().cuda()

In [1]:
class Trzecia(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        
        x = torch.randn(100, 100).view(-1, 1, 100, 100)
        self._to_linear = None
        self.convs(x)
        self.dropout1 = nn.Dropout()
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.dropout2 = nn.Dropout()
        self.fc2 = nn.Linear(512, 1)
        
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
        if self._to_linear is None:
            self._to_linear = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = self.dropout1(x)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x
    
model = Trzecia().cuda()
opt = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_function = nn.MSELoss()

NameError: name 'nn' is not defined